# Using Numpy efficiently

**Michiel Stock** [email](michiel.stock@ugent.be)

In [1]:
import numpy as np

## Vectorization

- *Python*: easy to use, but very slow (at lower level)

- *C*: very hard to use and learn, but extremely fast!

- *Numpy* is a python library implemented in C

> Try to avoid for-loops in favor for implementation in pure Numpy (**faster** + **cleaner**)!

### Example: implementing the gradient of logistic loss

$$
\nabla L(w) = \sum_{i=1}^n (y_i - \sigma_i)x_i
$$

In [2]:
# make some matrices
n, p = 1000, 100

X = np.random.randn(n, p)
y = np.random.binomial(1, 0.4, (n,))
sigma = np.random.rand(n)

In [3]:
def gradient_for_loop():
    grad = np.zeros((p, ))
    for i in range(n):
        xi = X[i,:]
        grad = grad + (y[i] - sigma[i]) * xi
    return grad

In [4]:
def gradient_vectorized():
    grad = X.T @ (y - sigma)
    return grad

In [5]:
gradient_for_loop()[:10]

array([ -2.72351694,  35.85100845, -22.19273948, -16.22745368,
         3.3854949 ,   6.91499947,  43.32664044, -16.83031321,
       -11.56459956,   9.72486346])

In [6]:
gradient_vectorized()[:10]

array([ -2.72351694,  35.85100845, -22.19273948, -16.22745368,
         3.3854949 ,   6.91499947,  43.32664044, -16.83031321,
       -11.56459956,   9.72486346])

In [7]:
%timeit gradient_for_loop()

100 loops, best of 3: 10.8 ms per loop


In [8]:
%timeit gradient_vectorized()

The slowest run took 8.68 times longer than the fastest. This could mean that an intermediate result is being cached.
10000 loops, best of 3: 45.9 µs per loop


## Broadcasting

Adding, multiplying matrices in Numpy do not need to be of the same shape = broadcasting of a matrix.

![Example of Broadcasting](Figures/broadcasting.png)

$$
\nabla L(w) = \sum_{i=1}^n x_i x_i^\top \sigma_i (1-\sigma_i)
$$

In [9]:
def hessian_for_loop():
    hess = np.zeros((p, p))
    for i in range(n):
        xi = X[i,:]
        sigma_i = sigma[i]
        hess = hess + xi.reshape((-1, 1)) @ xi.reshape((1, -1)) * sigma_i * (1 - sigma_i)
    return hess

In [10]:
def hessian_broadcasting():
    hess = (X.T * sigma * (1 - sigma)) @ X
    return hess

In [11]:
hessian_for_loop()[:5,:][:,:5]

array([[ 166.59012483,   -0.49219627,   -4.07980141,   -1.37499794,
         -10.61394254],
       [  -0.49219627,  159.17028568,    3.6362933 ,    1.03692227,
          -9.04167263],
       [  -4.07980141,    3.6362933 ,  164.83099477,   -6.87522001,
           6.57908363],
       [  -1.37499794,    1.03692227,   -6.87522001,  167.91092752,
         -10.5586751 ],
       [ -10.61394254,   -9.04167263,    6.57908363,  -10.5586751 ,
         169.80676125]])

In [12]:
hessian_broadcasting()[:5,:][:,:5]

array([[ 166.59012483,   -0.49219627,   -4.07980141,   -1.37499794,
         -10.61394254],
       [  -0.49219627,  159.17028568,    3.6362933 ,    1.03692227,
          -9.04167263],
       [  -4.07980141,    3.6362933 ,  164.83099477,   -6.87522001,
           6.57908363],
       [  -1.37499794,    1.03692227,   -6.87522001,  167.91092752,
         -10.5586751 ],
       [ -10.61394254,   -9.04167263,    6.57908363,  -10.5586751 ,
         169.80676125]])

In [13]:
%timeit hessian_for_loop()

10 loops, best of 3: 76.8 ms per loop


In [14]:
%timeit hessian_broadcasting()

100 loops, best of 3: 1.96 ms per loop


## Memory use

Initializing a matrix using `np.ones`, `np.zeros`, `np.random.rand` etc. or making a new matrix **consumes memory**.

> `x = x + v  #  make NEW matrix x`

> `x += v  #  update elements of x`

or, equivalently,

> `x[:] = x + v  #  update elements of x`

In [15]:
def replace_matrix(n_steps=100, size=(5000, 5000)):
    x = np.zeros(size)
    for i in range(n_steps):
        x = x + 1  # new matrix every step
    return x

In [16]:
def inplace_matrix(n_steps=100, size=(5000, 5000)):
    x = np.zeros(size)
    for i in range(n_steps):
        x += 1  # update elements IN matrix
    return x

In [17]:
%timeit replace_matrix()

1 loop, best of 3: 17 s per loop


In [18]:
%timeit inplace_matrix()

1 loop, best of 3: 2.58 s per loop
